## Imports

In [117]:
import numpy as np

from pylabnet.utils.logging.logger import LogClient
from pylabnet.utils.helper_methods import load_config
import pylabnet.utils.pulseblock.pulse as po
import pylabnet.utils.pulseblock.pulse_block as pb
from pylabnet.utils.pulseblock.pb_iplot import iplot
from pylabnet.utils.pulseblock.pb_sample import pb_sample
from pylabnet.hardware.awg.zi_hdawg import Driver, Sequence, AWGModule
from pylabnet.hardware.staticline import staticline
from pylabnet.utils.zi_hdawg_pulseblock_handler.zi_hdawg_pb_handler import DIOPulseBlockHandler
from pylabnet.network.client_server import dio_breakout
from pylabnet.network.client_server import si_tt_cnt_monitor

from pyvisa import VisaIOError, ResourceManager

from pylabnet.hardware.oscilloscopes.tektronix_tds2004C import Driver
from pylabnet.network.client_server.tektronix_tds2004C import Client

from pylabnet.gui.igui.iplot import SingleTraceFig, MultiTraceFig, StaggeredTraceFig


# Digital pulse playback using `pulseblock` and the DIO output of a ZI HDWAG

In [2]:
dev_id = 'dev8227'

# Instantiate logger.
logger = LogClient(
    host='140.247.189.94',
    port=45306,
    module_tag=f'ZI HDAWG {dev_id}'
)

# Instanciate HDAWG driver.
hd = Driver(dev_id, logger=logger)

Let's start with defining the digital part of a typical rabi-sequence. We use the `rabi_element` function as defined in the `pulseblock` demo notebook:

In [3]:
# Can be used to slow things down
scaling = 1

# Define the pulse sequence
test_pulse = pb.PulseBlock(
    p_obj_list=[
        po.PTrue(ch='green',  dur=1e-6*scaling),
        po.PTrue(ch='toptica', t0=2e-6*scaling,  dur=5e-6*scaling),
        po.PTrue(ch='gate', t0=1.9e-6*scaling, dur=5.2e-6*scaling)
    ]
)

# Add some fake pulses in the "readout" window
count_t0s = np.linspace(2.1*10**-6, 6.9*10**-6, 10)*scaling
for count_t0 in count_t0s:
    test_pulse.insert(
        po.PTrue(
            ch='fake_counts',
            dur=50*10**-9*scaling,
            t0=count_t0
        )
    )

# Default states (in future, module should auto default to False)
test_pulse.dflt_dict = dict(
    gate=po.DFalse(),
    green=po.DFalse(),
    toptica=po.DFalse(),
    fake_counts=po.DFalse()
)

# Plot
iplot(test_pulse)

Now, let's turn this `pulseblock` instance into an intruction set for the DIO outputs of the HDAWG. We want to get a series on `setDIO()` and `wait()` commands, which reproduce the pulse sequence.

In [4]:
# Load and verify assignment dict
assignment_dict = load_config('dio_assignment', logger=logger)
print(assignment_dict)

{'green': 31, 'toptica': 30, 'gate': 29, 'fake_counts': 28}


In [5]:
# Instanciate pulseblock handler.
pb_handler = DIOPulseBlockHandler(
    pb = test_pulse,
    assignment_dict=assignment_dict,
    hd=hd
)

In [6]:
# Generate .seqc instruction set which represents pulse sequence.
dig_pulse_sequence = pb_handler.get_dio_sequence()

In [7]:
# Let's have a look.
print(dig_pulse_sequence)

setDIO(2147483648);wait(295);setDIO(0);wait(265);setDIO(536870912);wait(26);setDIO(1610612736);wait(26);setDIO(1879048192);wait(12);setDIO(1610612736);wait(140);setDIO(1879048192);wait(12);setDIO(1610612736);wait(140);setDIO(1879048192);wait(12);setDIO(1610612736);wait(140);setDIO(1879048192);wait(12);setDIO(1610612736);wait(140);setDIO(1879048192);wait(13);setDIO(1610612736);wait(139);setDIO(1879048192);wait(12);setDIO(1610612736);wait(140);setDIO(1879048192);wait(13);setDIO(1610612736);wait(139);setDIO(1879048192);wait(13);setDIO(1610612736);wait(139);setDIO(1879048192);wait(12);setDIO(1610612736);wait(140);setDIO(1879048192);wait(13);setDIO(1610612736);wait(11);setDIO(536870912);wait(25);setDIO(0);wait(0);


This `.seqc` snippet can now be used in any sequence to reproduce the pulse sequence above. Let's try to write a simple sequence which outputs the digital pulse sequence as well as a trigger signal, and let's look at the results on the scope. 

To do so, connect marker output 1 of the HDWAG to the scope and use it as scope trigger, end then connect DIO pins 15, 17, and 31 each to one scope channel.

In [8]:
# This sequence will send out a trigger and execute the commands  dig_pulse (yet to be specified).
sequence_txt = """\

        while (1) {

          _dig_pulse_
          
          
          // Wait
          wait(1000);

        }
        """

# Create Sequence instance.
seq = Sequence(hd, sequence_txt, ['dig_pulse'])

# Replace dig_pulse placeholder by digital pulse sequence instruction set.
seq.replace_placeholder('dig_pulse', dig_pulse_sequence)

In [9]:
# Create an instance of the AWG Module.
awg = AWGModule(hd, 3)
awg.set_sampling_rate('2.4 GHz') # Set 2.4 GHz sampling rate.

# Upload sequence.
if awg is not None:
    awg.compile_upload_sequence(seq)

In [10]:
# Now we're almost ready, we only have to make sure that the 8 bit buses for bits 15, 17, and 31 are driven. 
# This can be done automatically by calling the following:

pb_handler.setup_hd()

In [3]:
# Connect to DIO breakout server (launched via launch control)
dio_ip = '140.247.189.82'
dio_port = 42657
dio_breakout_client = dio_breakout.Client(
    host = dio_ip,
    port = dio_port
)

In [12]:
# target_voltage = 0.3
# dio_breakout_client.set_high_voltage(1, 3, target_voltage)

In [4]:
for channel in range(4):
    print(f'Channel {channel}, low voltage: {dio_breakout_client.get_low_voltage(1, channel)}, '
          f'high voltage: {dio_breakout_client.get_high_voltage(1, channel)}')

Channel 0, low voltage: 0.0, high voltage: 5.0000762951094835
Channel 1, low voltage: 0.0, high voltage: 5.0000762951094835
Channel 2, low voltage: 0.0, high voltage: 5.0000762951094835
Channel 3, low voltage: 0.0, high voltage: 0.29999237048905164


In [118]:
for channel in range(4):
    print(f'Channel {channel}, low voltage: {dio_breakout_client.get_low_voltage(0, channel)}, '
          f'high voltage: {dio_breakout_client.get_high_voltage(0, channel)}')

Channel 0, low voltage: 0.0, high voltage: 4.9999237048905165
Channel 1, low voltage: 0.0, high voltage: 5.0000762951094835
Channel 2, low voltage: 0.0, high voltage: 5.0000762951094835
Channel 3, low voltage: 0.0, high voltage: 4.9999237048905165


In [121]:
target_voltage = 3
dio_breakout_client.set_high_voltage(0, 0, target_voltage)

0

In [14]:
# Now we're ready, start the Start the AWG
awg.start()

In [ ]:
# TODO: TEST COUNTERS

In [15]:
awg.stop()

# Staticline stuff

In [51]:

snspd1 = staticline.Driver(
    name='snspd1',
    logger=logger,
    hardware_module=hd,
    DIO_bit=24,
) 


snspd2 = staticline.Driver(
    name='snspd1',
    logger=logger,
    hardware_module=hd,
    DIO_bit=27,
) 

In [218]:
snspd1.up()

In [216]:
snspd1.down()

In [177]:
snspd2.up()

In [168]:
snspd2.down()

In [219]:
target_voltage = 4
dio_breakout_client.set_high_voltage(0, 0, target_voltage)

0

In [161]:
target_voltage = 4
dio_breakout_client.set_high_voltage(0, 3, target_voltage)

0

In [107]:
dio_breakout_client.get_high_voltage(0, 0)

1.3998626688029296

In [ ]:

scope = Client(
    host='localhost',
    port=2253
)

In [171]:
# Let assume we have a shutter connected to the HDAWG DIO-pin 1 which 
# is open if the pin voltage is high and closed otherwise.

aom = staticline.Driver(
    name='AOM',
    logger=logger,
    hardware_module=hd,
    DIO_bit=30,
) 

In [206]:
aom.up()


In [190]:
aom.down()

In [173]:
# AOM Switch 
import pylabnet.hardware.ni_daqs.nidaqmx_card as nidaqmx
import pylabnet.network.client_server.nidaqmx_card as nidaqmx_card_server

In [181]:
device_name = 'PXI1Slot4'

daq = nidaqmx.Driver(device_name=device_name, logger=logger)

In [210]:
daq.set_ao_voltage('ao2', 0)

In [215]:
daq.set_ao_voltage('ao2', 0.05)

In [194]:
aom.down()

In [197]:
aom.up()

In [188]:
daq.set_ao_voltage('ao0',6.92)